In [1]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle

In [2]:
data=pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
##preprocess the data
## drop the irrelevant columns..
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [4]:
## encoding categorical variables
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [5]:
## one hot encoding
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo=OneHotEncoder()
geo_encoder=onehot_encoder_geo.fit_transform(data[['Geography']])


In [6]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [7]:
geo_encoded_df=pd.DataFrame(geo_encoder.toarray(),columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [8]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [9]:
## combine all the columns with the original data 
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [10]:
## split the data into features and target
X=data.drop('EstimatedSalary',axis=1)
y=data['EstimatedSalary']


In [11]:
## split the dataset into training and testing sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)


## scaling these features

scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [12]:
## save the encoder or scaler
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

## ANN regression problem statement

In [13]:

import tensorflow as tf  
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 

In [14]:
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), ## HL1 connected with input layer
    Dense(32,activation='relu'),  ## HL2
    Dense(1)  ## Output layer
])

model.compile(optimizer='adam',loss='mean_absolute_error',metrics='mae')
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

# set up Tensorboard

log_dir="regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)


In [17]:
# set up Early Stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)


In [18]:
## train the model
history =model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorboard_callback,early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 100387.6719 - mae: 100387.6719 - val_loss: 98552.7656 - val_mae: 98552.7656
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 99758.0234 - mae: 99758.0234 - val_loss: 97268.8438 - val_mae: 97268.8438
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 97459.4219 - mae: 97459.4219 - val_loss: 93825.5938 - val_mae: 93825.5938
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 92775.4766 - mae: 92775.4766 - val_loss: 87926.4688 - val_mae: 87926.4688
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 85776.5078 - mae: 85776.5078 - val_loss: 80105.4766 - val_mae: 80105.4766
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 77232.2344 - mae: 77232.2344 - val_loss: 71489.8984 - val_mae: 71489.8984
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 68517.6875 - mae: 68517.6875 

In [19]:
%load_ext tensorboard

In [21]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6008 (pid 38313), started 0:00:06 ago. (Use '!kill 38313' to kill it.)

In [22]:
#evalute the model
test_loss,test_mae=model.evaluate(X_test,y_test)

63/63 [==============================] - 0s 558us/step - loss: 50301.6680 - mae: 50301.6680


In [23]:
print(test_mae)

50301.66796875


In [24]:
print(test_loss)

50301.66796875


In [25]:
model.save('regression.h5')

/Users/vijaykumar/Desktop/DeepLearning/venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
